In [1]:
import argparse
import numpy as np

from experiments_comp.utils_comp import generate_dataset
from experiments_comp.invase_regression import invase
from experiments_comp.utils import feature_performance_metric, prediction_performance_metric   
from sklearn.metrics import mean_squared_error
from experiments.utils import *

In [2]:
# parser for invase model and data generation

parser = argparse.ArgumentParser()
parser.add_argument(
  '--data_type',
  choices=['syn1','syn2','syn3','syn4','syn5','syn6', 'syn7', 'syn8'],
  default='syn7',
  type=str)
parser.add_argument(
  '--train_no',
  help='the number of training data',
  default=50000,
  type=int)
parser.add_argument(
  '--test_no',
  help='the number of testing data',
  default=100000,
  type=int)
parser.add_argument(
  '--dim',
  help='the number of features',
  choices=[11, 100],
  default=50,
  type=int)
parser.add_argument(
  '--lamda',
  help='inavse hyper-parameter lambda',
  default=0.1,
  type=float)
parser.add_argument(
  '--actor_h_dim',
  help='hidden state dimensions for actor',
  default=100,
  type=int)
parser.add_argument(
  '--critic_h_dim',
  help='hidden state dimensions for critic',
  default=200,
  type=int)
parser.add_argument(
  '--n_layer',
  help='the number of layers',
  default=3,
  type=int)
parser.add_argument(
  '--batch_size',
  help='the number of samples in mini batch',
  default=1000,
  type=int)
parser.add_argument(
  '--iteration',
  help='the number of iteration',
  default=4500,
  type=int)
parser.add_argument(
  '--activation',
  help='activation function of the networks',
  choices=['selu','relu'],
  default='relu',
  type=str)
parser.add_argument(
  '--learning_rate',
  help='learning rate of model training',
  default=0.00001,
  type=float)
parser.add_argument(
  '--model_type',
  help='inavse or invase- (without baseline)',
  choices=['invase','invase_minus'],
  default='invase_minus',
  type=str)

args = parser.parse_args([]) 

model_parameters = {'lamda': args.lamda,
                  'actor_h_dim': args.actor_h_dim, 
                  'critic_h_dim': args.critic_h_dim,
                  'n_layer': args.n_layer,
                  'batch_size': args.batch_size,
                  'iteration': args.iteration, 
                  'activation': args.activation, 
                  'learning_rate': args.learning_rate}

In [4]:
# To compute the metric P-MSE

def return_output(x, coefs):
    logit1 = np.sum(coefs[0:2] * x[:, 0:2], axis=1)
    logit2 = np.sum(coefs[2:4] * x[:, 2:4], axis=1)
    idx1 = (x[:, 4] < 0) * 1
    idx2 = (x[:, 4] >= 0) * 1
    logit = logit1 * idx1 + logit2 * idx2
    return logit

def sample_mcexp(X, coefs, mean, cov, S, N=1000):
    d = X.shape[1]
    index = list(range(d))
    y_cond = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        x = X[i]
        rg_data = np.zeros(shape=(N, d))
        rg_data[:, S] = x[S]
        S_bar = [i for i in index if i not in S]
        
        if len(S) != d:
            rg = sampleMVN(N, mean, cov, S_bar, S, x[S])
            rg_data[:, S_bar] = rg
            y_cond[i] = np.mean(return_output(rg_data, coefs))
        else:
            y_cond[i] = np.mean(return_output(rg_data, coefs))
            
    return y_cond

# Generate toy dataset

In [5]:
# Generate dataset
d = args.dim
p = 0.8
mean = np.zeros(d)
coefs = np.ones(d)
# Determinitist covariance
cov = p*np.ones(shape=(d, d)) + 5*np.eye(d)
# b = np.random.randn(d, d)
# cov = np.dot(b, b.T)

x_train, y_train, g_train = generate_dataset (mean, cov, n = args.train_no, 
                                            dim = args.dim, 
                                            data_type = args.data_type, 
                                            seed = 0, reg=True, coefs=coefs)

x_test, y_test, g_test = generate_dataset (mean, cov, n = args.test_no,
                                         dim = args.dim, 
                                         data_type = args.data_type, 
                                         seed = 5, reg=True, coefs=coefs)

In [6]:
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)

# Model 1: INVASE

In [7]:
# # # # Train the model
model = invase(x_train, y_train, args.model_type, model_parameters)

model.train(x_train, y_train)    

Iterations: 0, critic accuracy: 14.7167329788208, actor loss: -391.5284
Iterations: 100, critic accuracy: 13.201102256774902, actor loss: -299.1104
Iterations: 200, critic accuracy: 11.990948677062988, actor loss: -287.3979
Iterations: 300, critic accuracy: 11.222001075744629, actor loss: -285.506
Iterations: 400, critic accuracy: 10.672489166259766, actor loss: -250.1972
Iterations: 500, critic accuracy: 10.266728401184082, actor loss: -227.2632
Iterations: 600, critic accuracy: 9.946744918823242, actor loss: -229.9763
Iterations: 700, critic accuracy: 9.681632995605469, actor loss: -240.4062
Iterations: 800, critic accuracy: 9.451498985290527, actor loss: -224.4285
Iterations: 900, critic accuracy: 9.244540214538574, actor loss: -221.4833
Iterations: 1000, critic accuracy: 9.058870315551758, actor loss: -220.0287
Iterations: 1100, critic accuracy: 8.89184856414795, actor loss: -206.8741
Iterations: 1200, critic accuracy: 8.73809814453125, actor loss: -208.1987
Iterations: 1300, criti

## Discovery Score

In [8]:
## Evaluation
# Compute importance score
g_hat = model.importance_score(x_test)[:1000]
importance_score = 1.*(g_hat > 0.5)

# Evaluate the performance of feature importance
mean_tpr, std_tpr, mean_fdr, std_fdr = \
feature_performance_metric(g_test[:1000], importance_score)

# Print the performance of feature importance    
print('TPR mean: ' + str(np.round(mean_tpr,1)) + '\%, ' + \
    'TPR std: ' + str(np.round(std_tpr,1)) + '\%, ')
print('FDR mean: ' + str(np.round(mean_fdr,1)) + '\%, ' + \
    'FDR std: ' + str(np.round(std_fdr,1)) + '\%, ')

# Predict labels
y_hat = model.predict(x_test)

mse = mean_squared_error(y_test, y_hat)

# Print the performance    
print('mse = {}'.format(mse))

TPR mean: 100.0\%, TPR std: 0.0\%, 
FDR mean: 87.4\%, FDR std: 1.3\%, 
mse = 2.6380980900041426


## P-MSE Score

In [9]:
importance = importance_score.copy()
N = 1000
allS = [i for i in range(x_train.shape[1])]
n_test = 1000
err_samples = []
for i in tqdm(range(n_test)):
    S = np.argwhere(importance[i] == 1).reshape(-1)
    err = (sample_mcexp(x_test[i:i+1], coefs, mean, cov, S=S, N=N) -\
           sample_mcexp(x_test[i:i+1], coefs, mean, cov, S=allS, N=N))**2
    err_samples.append(err)

100%|██████████| 1000/1000 [00:01<00:00, 578.93it/s]


In [10]:
np.mean(err_samples)

0.0

# Model 2: SDP explanations via RF

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
# from skranger.ensemble.classifier import RangerForestClassifier
from sklearn.ensemble import RandomForestRegressor
from acv_explainers.acv_tree import ACVTreeAgnostic
# Necessary packages
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
import pandas as pd

In [8]:
n = args.train_no
# y_train = np.argmax(y_train, axis=1)
# y_test = np.argmax(y_test, axis=1)
y_train = y_train.squeeze()
y_test = y_test.squeeze()
x_train_rf = x_train[:10000]
y_train_rf = y_train[:10000].squeeze()
# y_test_rf = np.argmax(y_test, axis=1)

In [9]:
model = RandomForestRegressor(n_estimators=10, max_depth=8)
model.fit(x_train, y_train)

RandomForestRegressor(max_depth=8, n_estimators=10)

In [10]:
# Evaluate the performance 
y_hat = model.predict(x_test)
mse = mean_squared_error(y_test, y_hat)

# Print the performance    
print('mse = {}'.format(mse))

mse = 0.22896751013859182


In [11]:
max_size = 10000
x_train = pd.DataFrame(x_train[:max_size], columns=['X{}'.format(i) for i in range(x_train.shape[1])])
x_test = pd.DataFrame(x_test[:max_size], columns=['X{}'.format(i) for i in range(x_test.shape[1])])
y_test = pd.DataFrame(y_test[:max_size])
y_train = pd.DataFrame(y_train[:max_size])

100%|██████████| 10/10 [00:00<00:00, 13.99it/s]


In [13]:
# Build ACVTree

acvtree = ACVTreeAgnostic(model, x_train.values)

100%|██████████| 10/10 [00:00<00:00, 14.26it/s]


In [19]:
# SDP Explanations
sdp_global, sdp_index, len_sdp, sdp = acvtree.importance_sdp_rf(x_test.values[:1000].astype(np.double), y_test.values.reshape(-1)[:1000].astype(np.double),
                                                                x_train.values.astype(np.double), y_train.values.reshape(-1).astype(np.double),
                                                                stop=False, pi_level=0.9, classifier=0, t=5, minimal=2)

100%|██████████| 9/9 [1:28:43<00:00, 591.51s/it]   


In [20]:
sdp_index

array([[ 4,  3,  2, ..., -1, -1, -1],
       [ 4,  0,  1, ..., -1, -1, -1],
       [ 4,  0,  1, ..., -1, -1, -1],
       ...,
       [ 4,  0,  1, ..., -1, -1, -1],
       [ 4,  0,  1, ..., -1, -1, -1],
       [ 4,  3,  2, ..., -1, -1, -1]])

In [22]:
def return_sdp_importance(sdp_index):
    fi = np.zeros(shape=sdp_index.shape)
    for i in range(sdp_index.shape[0]):
        for j in range(sdp_index.shape[1]):
            if sdp_index[i, j] != -1:
                fi[i, sdp_index[i, j]] = 1 
    return fi

In [23]:
importance_sdp = return_sdp_importance(sdp_index)

## Discovery Score

In [25]:
# Evaluate the performance of feature importance
mean_tpr, std_tpr, mean_fdr, std_fdr = \
feature_performance_metric(g_test, importance_sdp)

# Print the performance of feature importance    
print('TPR mean: ' + str(np.round(mean_tpr,1)) + '\%, ' + \
    'TPR std: ' + str(np.round(std_tpr,1)) + '\%, ')
print('FDR mean: ' + str(np.round(mean_fdr,1)) + '\%, ' + \
    'FDR std: ' + str(np.round(std_fdr,1)) + '\%, ')

TPR mean: 99.5\%, TPR std: 5.1\%, 
FDR mean: 2.2\%, FDR std: 10.7\%, 


## P-MSE Score

In [27]:
importance = importance_sdp.copy()
N = 1000
allS = [i for i in range(x_train.shape[1])]
n_test = 1000
err_samples = []
for i in tqdm(range(n_test)):
    S = np.argwhere(importance[i] == 1).reshape(-1)
    err = (sample_mcexp(x_test.values[i:i+1], coefs, mean, cov, S=S, N=N) -\
           sample_mcexp(x_test.values[i:i+1], coefs, mean, cov, S=allS, N=N))**2
    err_samples.append(err)

np.mean(err_samples)

100%|██████████| 1000/1000 [00:02<00:00, 450.08it/s]


0.021894446936767276

# Model 3: SHAP

In [28]:
columns = ['X{}'.format(i) for i in range(x_test.shape[1])]

In [29]:
import shap
import matplotlib.pyplot as plt

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_test)

In [31]:
K = np.sum(g_test, axis=1).astype(np.int)

array([3, 3, 3, ..., 3, 3, 3])

In [32]:
n_test = x_test.shape[0]

In [33]:
importance_shap = np.zeros((n_test, d))
K = np.sum(g_test, axis=1).astype(np.int)
for i in range(n_test):
    argsort = np.argsort(-np.abs(shap_values[i]))
    for j in range(K[i]):
        importance_shap[i, argsort[j]] = 1
    

## Discovery Score

In [34]:
# Evaluate the performance of feature importance
mean_tpr, std_tpr, mean_fdr, std_fdr = \
feature_performance_metric(g_test, importance_shap)

# Print the performance of feature importance    
print('TPR mean: ' + str(np.round(mean_tpr,1)) + '\%, ' + \
    'TPR std: ' + str(np.round(std_tpr,1)) + '\%, ')
print('FDR mean: ' + str(np.round(mean_fdr,1)) + '\%, ' + \
    'FDR std: ' + str(np.round(std_fdr,1)) + '\%, ')

TPR mean: 73.1\%, TPR std: 19.8\%, 
FDR mean: 26.9\%, FDR std: 19.8\%, 


## P-MSE Score

In [35]:
importance = importance_shap.copy()
N = 1000
allS = [i for i in range(x_train.shape[1])]
n_test = 1000
err_samples = []
for i in tqdm(range(n_test)):
    S = np.argwhere(importance[i] == 1).reshape(-1)
    err = (sample_mcexp(x_test.values[i:i+1], coefs, mean, cov, S=S, N=N) -\
           sample_mcexp(x_test.values[i:i+1], coefs, mean, cov, S=allS, N=N))**2
    err_samples.append(err)

np.mean(err_samples)

100%|██████████| 1000/1000 [00:02<00:00, 419.68it/s]


0.7945839156430332

# Model 4: LIME

In [36]:
np.random.seed(1)

In [37]:
import lime
import lime.lime_tabular

In [38]:
c = [i for i in range(x_train.shape[1])]

In [39]:
explainer = lime.lime_tabular.LimeTabularExplainer(x_train.values, feature_names=c, verbose=True, mode='regression')

In [40]:
from tqdm import tqdm

In [41]:
list_exp = []
n_test = 1000
for i in tqdm(range(n_test)):
    exp = explainer.explain_instance(x_test.values[i], model.predict, num_features=3)
    list_exp.append([ex[0] for ex in exp.local_exp[0]])

  0%|          | 1/1000 [00:15<4:25:28, 15.94s/it]

Intercept 1.085397464063249
Prediction_local [-2.86558098]
Right: -2.931161197577681


  0%|          | 2/1000 [00:31<4:23:17, 15.83s/it]

Intercept 0.5628388377107446
Prediction_local [-1.66058732]
Right: -2.0591819549379613


  0%|          | 3/1000 [00:47<4:19:45, 15.63s/it]

...

Intercept 1.2958791568390369
Prediction_local [-3.70211185]
Right: -6.819554322765645


100%|█████████▉| 999/1000 [4:13:16<00:15, 15.01s/it]

Intercept 1.3550314173128422
Prediction_local [-3.7719575]
Right: -5.321223687507614


100%|██████████| 1000/1000 [4:13:31<00:00, 15.21s/it]

Intercept 0.25618701631572965
Prediction_local [-0.57079587]
Right: -0.031102226535924354


In [42]:
importance_lime = np.zeros((n_test, d))
K = np.sum(g_test, axis=1).astype(np.int)
for i in range(n_test):
    for j in range(K[i]):
        importance_lime[i, list_exp[i][j]] = 1
    

In [43]:
importance_lime

array([[1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

## Discovery Score

In [44]:
# Evaluate the performance of feature importance
mean_tpr, std_tpr, mean_fdr, std_fdr = \
feature_performance_metric(g_test, importance_lime)

# Print the performance of feature importance    
print('TPR mean: ' + str(np.round(mean_tpr,1)) + '\%, ' + \
    'TPR std: ' + str(np.round(std_tpr,1)) + '\%, ')
print('FDR mean: ' + str(np.round(mean_fdr,1)) + '\%, ' + \
    'FDR std: ' + str(np.round(std_fdr,1)) + '\%, ')

TPR mean: 50.7\%, TPR std: 16.7\%, 
FDR mean: 49.3\%, FDR std: 16.7\%, 


## P-MSE Score

In [46]:
importance = importance_lime.copy()
N = 1000
allS = [i for i in range(x_train.shape[1])]
n_test = 1000
err_samples = []
for i in tqdm(range(n_test)):
    S = np.argwhere(importance[i] == 1).reshape(-1)
    err = (sample_mcexp(x_test.values[i:i+1], coefs, mean, cov, S=S, N=N) -\
           sample_mcexp(x_test.values[i:i+1], coefs, mean, cov, S=allS, N=N))**2
    err_samples.append(err)

np.mean(err_samples)

100%|██████████| 1000/1000 [00:02<00:00, 437.94it/s]


5.005296546757587